In [ ]:
import pandas as pd
import numpy as np
import json, os, glob, random
import torch
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
def xavier_(weights):
    for weight in weights:
        in_dim, out_dim = weight.shape[-2:]
        np.copyto(dst=weight, src=np.random.randn(*weight.shape) * np.sqrt(2. / (in_dim + out_dim)))

In [ ]:
def eval_accuracy(model, val, y_val):
    output = model(val)
    y_hat = np.argmax(output, axis=1)
    return accuracy_score(y_val, y_hat)

In [ ]:
def relu(x):
    return np.maximum(x, 0)

In [ ]:
class RELUFunction():
    def forward(self, x):
        self.a = relu(x)
        return self.a

    def backward(self, grad):
        return 1. * (x > 0) * grad.reshape(self.a.shape)

    def __call__(self, x):
        return self.forward(x)

In [ ]:
class MSELoss:
    def __init__(self, eps=1e-15):
        self.eps = eps

    def forward(self, output, target):
        self.target = target.squeeze()
        self.output = output.squeeze()
        return ((self.target - self.output)**2).mean()

    def backward(self):
        return self.output - self.target

    def __call__(self, output, target):
        return self.forward(output, target)

In [ ]:
def Linear(in_dim, out_dim):
    W = np.random.normal(loc=0, scale=0.1, size=(in_dim, out_dim))
    b = np.random.normal(loc=0, scale=0.1, size=(1, out_dim))
    return W, b

In [ ]:
class LinearLayer():
    def __init__(self, in_dim, out_dim):
        self.W, self.b = Linear(in_dim, out_dim)

    def forward(self, x):
        self.data = x
        z = np.dot(x, self.W) + self.b
        return z

    def backward(self, grad):
        dW = np.dot(self.data.T, grad)
        db = grad.sum(axis=0)
        dx = np.dot(grad, self.W.T)
        return dW, db, dx

    def __call__(self, x):
        return self.forward(x)

    def parameters(self):
        return self.W, self.b

In [ ]:
class Dropout:
    def __init__(self, p=0.5):
        self.p = 1. - p

    def forward(self, x, training):
        if training:
            self.mask = np.random.binomial(1, self.p, size=x.shape)
            res = x * self.mask / self.p
            return res.reshape(x.shape)
        else:
            return x

    def backward(self, grad):
        return grad * self.mask / self.p # you need to scale signal

    def __call__(self, x, training):
        return self.forward(x, training)

In [ ]:
class Adam:
    def __init__(self, alpha=0.1, beta1=0.9, beta2=0.999, eps=1e-8, weight_decay=0.01):
        self.alpha = alpha
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.weight_decay = weight_decay
        self.t = 0

    def step(self, grads, params):
        if self.t == 0:
            self.m = {k: np.zeros_like(v) for k, v in enumerate(params)}
            self.v = {k: np.zeros_like(v) for k, v in enumerate(params)}
        self.t += 1
        for k, (param, grad) in enumerate(zip(params, grads)):
            self.m[k] = self.beta1*self.m[k] + (1 - self.beta1)*grad
            self.v[k] = self.beta2*self.v[k] + (1 - self.beta2)*(grad**2)
            m_hat = self.m[k] / (1 - self.beta1**self.t)
            v_hat = self.v[k] / (1 - self.beta2**self.t)
            param -= self.alpha * (m_hat / (np.sqrt(v_hat) + self.eps) + self.weight_decay*param)
        return params

In [ ]:
class FeedforwardNetwork:
    def __init__(self):
        self.fc1 = LinearLayer(36, 32)
        self.dp1 = Dropout(0.1) ###
        self.fc2 = LinearLayer(32, 16)
        self.dp2 = Dropout(0.1) ###
        self.fc3 = LinearLayer(16, 4)
        self.dp3 = Dropout(0.1) ###
        self.relu = RELUFunction()

        xavier_(self.fc1.parameters())
        xavier_(self.fc2.parameters())
        xavier_(self.fc3.parameters())

        self.training = True ###

    def forward(self, x):
        self.data = x
        z1 = self.fc1(x)
        z1 = self.dp1.forward(z1, self.training) ###
        self.a1 = self.relu(z1)
        z2 = self.fc2(self.a1)
        z2 = self.dp2.forward(z2, self.training) ###
        self.a2 = relu(z2)
        z3 = self.fc3(self.a2)
        z3 = self.dp3.forward(z3, self.training) ###
        self.a3 = relu(z3)
        return self.a2

    def gradients(self):
        dz3 = self.loss_function.backward()
        dz3 = self.dp3.backward(dz3) ###
        dW3, db3, dx3 = self.fc3.backward(dz3)
        dz2 = self.relu.backward(dz3)
        dz2 = self.dp2.backward(dz2) ###
        dW2, db2, dx2 = self.fc2.backward(dz2)
        dz1 = self.relu.backward(dx2)
        dz1 = self.dp1.backward(dz1) ###
        dW1, db1, _ = self.fc1.backward(dz1)
        return (dW1, db1, dW2, db2)

    def backward(self, loss_function):
        self.loss_function = loss_function
        return self.gradients()

    def parameters(self):
        return self.fc1.parameters() + self.fc2.parameters() + self.fc3.parameters()

    def __call__(self, x):
        return self.forward(x)

    def size(self):
        s = 0
        for param in self.parameters():
            s += param.size
        return s

    def eval(self): ###
        self.training = False ###

In [ ]:
model = FeedforwardNetwork()
loss_function = MSELoss()
optimizer = Adam(alpha=0.001)

In [ ]:
# Read onell runtime data

pd.set_option('display.max_columns', None)

df_onell = pd.DataFrame()

path = './new_logs'

for file_name in glob.iglob(path + '/**/**.csv', recursive=True):
    #temp = pd.read_json(file_name)
    temp = pd.read_csv(file_name)
    df_onell = df_onell.append(temp, ignore_index = True)

df_onell = df_onell.astype(float)
df_onell = df_onell.drop(columns=['Unnamed: 0'])

In [ ]:
# Select rows with min average runtime

mean_runtime = df_onell.groupby(['n', 'dummy', 'epi', 'neu', 'rug', 'lambdaOne', 'lambda2', 'crossover', 'mutation'])['runtime'].transform('mean')
df_onell['runtime_mean'] = mean_runtime

idx = df_onell.groupby(['n', 'dummy', 'epi', 'neu', 'rug'])['runtime_mean'].transform('max') == df_onell['runtime_mean']
df_onell = df_onell[idx]

In [ ]:
# Read flacco data

# To fix the probem with the way output data is structured after multiple runs of onell

with open('flacco_logs.txt') as raw_file:
    with open('flacco_logs_fixed.txt', 'wt') as json_file:
        for line in raw_file:
            json_file.write(line.replace('}[', '}\n,'))

# Read json data and convert to pandas dataframe

df_flacco = None

with open('flacco_logs_fixed.txt') as json_file:
    
    data = json.load(json_file)
    
    df_flacco = pd.DataFrame.from_records(data)

    # features from flacco

    flacco_features = df_flacco['flacco features'].apply(pd.Series)
    df_flacco = pd.concat([df_flacco.drop('flacco features', axis=1), flacco_features], axis=1)
    df_flacco = df_flacco.astype(float)

In [ ]:
df_new = pd.merge(df_onell, df_flacco, on=['n', 'dummy', 'epi', 'neu', 'rug'])

In [ ]:
# Preprocess data before trainig process

df_new = df_new.astype(float)
df_new = df_new.fillna(0)
#df_new = (df_new - df_new.mean()) / df_new.std()
#df_new = df_new.fillna(0)

X = df_new.drop(['n', 'dummy', 'epi', 'neu', 'rug', 'lambdaOne', 'lambda2', 'crossover', 'mutation', 'runtime', 'runtime over n', 'runtime_mean'], axis=1)
X = (X - X.mean()) / X.std()
X = X.fillna(0)

y = df_new[['lambdaOne', 'lambda2', 'crossover', 'mutation']]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
num_epochs = 50
losses = []
accuracies = []

for epoch in range(num_epochs):
    loss_sum = 0

    inputs = X_train
    targets = y_train
    outputs = model(inputs)
    loss = loss_function(outputs, targets)
    loss_sum += loss
    grads = model.backward(loss_function)
    params = model.parameters()
    optimizer.step(grads, params)
    acc = eval_accuracy(model, X_val, y_val)
    losses.append(loss_sum)
    accuracies.append(acc)

In [ ]:
# Read flacco test data

# To fix the probem with the way output data is structured after multiple runs of onell

with open('flacco_data/test-data.txt') as raw_file:
    with open('flacco_data/test-data_fixed.txt', 'wt') as json_file:
        for line in raw_file:
            json_file.write(line.replace('}[', '}\n,'))

# Read json data and convert to pandas dataframe

df_flacco_test = None

with open('flacco_data/test-data_fixed.txt') as json_file:
    
    data_test = json.load(json_file)
    
    df_flacco_test = pd.DataFrame.from_records(data_test)

    # features from flacco

    flacco_features_test = df_flacco_test['flacco features'].apply(pd.Series)
    df_flacco_test = pd.concat([df_flacco_test.drop('flacco features', axis=1), flacco_features_test], axis=1)
    df_flacco_test = df_flacco_test.astype(float)

In [ ]:
# Preprocess data before trainig process

df_flacco_test = df_flacco_test.astype(float)
df_flacco_test = df_flacco_test.fillna(0)

X_test = df_flacco_test.drop(['n', 'dummy', 'epi', 'neu', 'rug'], axis=1)
X_test = (X_test - X_test.mean()) / X_test.std()
X_test = X_test.fillna(0)

In [ ]:
val = X_test
res = model.forward(val)

buf_res = pd.DataFrame({'lambdaOne': res[:, 0], 'lambda2': res[:, 1], 'crossover': res[:, 2], 'mutation': res[:, 3]})
full_res = pd.concat([df_flacco_test, buf_res], axis=1)

In [ ]:
with open('model_results.txt', 'w') as write_file:
    for index, row in full_res.iterrows():
        lambdaOne = row['lambdaOne'] if row['lambdaOne'] > 1 and row['lambdaOne'] < 10.0 else random.uniform(2.0, 9.0)
        lambda2 = row['lambda2'] if row['lambda2'] > 1 and row['lambda2'] < 10.0 else random.uniform(2.0, 9.0)
        crossover = row['crossover'] if row['crossover'] > 0 and row['crossover'] < 0.25 else random.uniform(0.01, 0.09)
        mutation = row['mutation'] if row['mutation'] > 1 and row['mutation'] < 10.0 else random.uniform(1.0, 9.0)
        write_file.write('{{"n": {}, "dummy": {}, "epi": {}, "neu": {}, "rug": {}, "lambdaOne":{}, "lambda2":{}, "crossover":{}, "mutation":{}}},\n'.format(
            row['n'], row['dummy'], row['epi'], row['neu'], row['rug'], row['lambdaOne'], row['lambda2'], row['crossover'], row['mutation']))